# Tworzymy nazwane klastry

W lekcji tej użyjemy LLM-ów do pomocy przy znalezieniu odpowiednich nazw i opisów dla klastrów.

In [1]:
import json
import pandas as pd
from openai import OpenAI
from dotenv import dotenv_values
from pycaret.clustering import predict_model, load_model

In [2]:
env = dotenv_values(".env")

openai_client = OpenAI(api_key=env["OPENAI_API_KEY"])

ładujemy nasze dane

In [3]:
df = pd.read_csv('welcome_survey_simple_v2.csv', sep=';')
df.head()

,age,edu_level,fav_animals,fav_place,gender
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna
2,45-54,Wyższe,Psy,W lesie,Mężczyzna
3,35-44,Średnie,Koty,W górach,Mężczyzna
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna


ładujemy model wytrenowany w poprzedniej lekcji

In [4]:
kmeans_pipeline = load_model('welcome_survey_clustering_pipeline_v2')

Transformation Pipeline and Model Successfully Loaded


aplikujemy model do danych

In [5]:
df_with_clusters = predict_model(model=kmeans_pipeline, data=df)
df_with_clusters["Cluster"].value_counts()

Cluster
Cluster 1    50
Cluster 7    39
Cluster 4    31
Cluster 2    27
Cluster 5    26
Cluster 0    22
Cluster 6    18
Cluster 3    16
Name: count, dtype: int64

stworzymy teraz prompt, który prześlemy do LLM-a w celu znalezienia odpowiednich nazw i opisów dla klastrów

In [6]:
cluster_descriptions = {} # pusty słownik
for cluster_id in df_with_clusters['Cluster'].unique(): # iterujemy po nazwach clustrów, gdzie każdy nazywamy zmienna id
    cluster_df = df_with_clusters[df_with_clusters['Cluster'] == cluster_id] # bierzemy tylko to rzędy, które sa skatogeryzowane, jako konkretne clustry
    summary = "" # zmienna z pustym łańcuchem znaków
    for column in df_with_clusters: # iterujemy po wszystkich kolumnach w naszym clustrze
        if column == 'Cluster': # pomijając kolumnę cluster- ją pomijamy
            continue

        value_counts = cluster_df[column].value_counts() # liczymy ilość wystąpiений w kolumnie w ramach danego clustra
        value_counts_str = ', '.join([f"{idx}: {cnt}" for idx, cnt in value_counts.items()]) # łączymy te wszystkie wartości
        summary += f"{column} - {value_counts_str}\n"

    cluster_descriptions[cluster_id] = summary

In [7]:
print(cluster_descriptions["Cluster 5"])

age - 35-44: 11, 45-54: 7, 55-64: 4, 18-24: 1, <18: 1, >=65: 1, unknown: 1, 25-34: 0
edu_level - Wyższe: 25, Podstawowe: 1, Średnie: 0
fav_animals - Inne: 13, Brak ulubionych: 9, Koty i Psy: 4, Koty: 0, Psy: 0
fav_place - Nad wodą: 15, Inne: 1, W górach: 0, W lesie: 0
gender - Mężczyzna: 17, Kobieta: 8



In [8]:
# tworzenie prompta dla llm- to jest tekst, który przekazujemy do modelu- lepiej działa, jak mu się podaje przykłady
prompt = "Użyliśmy algorytmu klastrowania."
for cluster_id, description in cluster_descriptions.items():
    prompt += f"\n\nKlaster {cluster_id}:\n{description}"

prompt += """
Wygeneruj najlepsze nazwy dla każdego z klasterów oraz ich opisy

Użyj formatu JSON. Przykładowo:
{
    "Cluster 0": {
        "name": "Klaster 0",
        "description": "W tym klastrze znajdują się osoby, które..."
    },
    "Cluster 1": {
        "name": "Klaster 1",
        "description": "W tym klastrze znajdują się osoby, które..."
    }
}
"""
print(prompt)

Użyliśmy algorytmu klastrowania.

Klaster Cluster 5:
age - 35-44: 11, 45-54: 7, 55-64: 4, 18-24: 1, <18: 1, >=65: 1, unknown: 1, 25-34: 0
edu_level - Wyższe: 25, Podstawowe: 1, Średnie: 0
fav_animals - Inne: 13, Brak ulubionych: 9, Koty i Psy: 4, Koty: 0, Psy: 0
fav_place - Nad wodą: 15, Inne: 1, W górach: 0, W lesie: 0
gender - Mężczyzna: 17, Kobieta: 8


Klaster Cluster 7:
age - 45-54: 13, 35-44: 10, 18-24: 8, 25-34: 7, >=65: 1, 55-64: 0, <18: 0, unknown: 0
edu_level - Średnie: 39, Podstawowe: 0, Wyższe: 0
fav_animals - Psy: 18, Koty: 9, Inne: 8, Brak ulubionych: 4, Koty i Psy: 0
fav_place - W górach: 18, Nad wodą: 12, Inne: 3, W lesie: 3
gender - Mężczyzna: 33, Kobieta: 6


Klaster Cluster 4:
age - 45-54: 31, 18-24: 0, 25-34: 0, 35-44: 0, 55-64: 0, <18: 0, >=65: 0, unknown: 0
edu_level - Wyższe: 31, Podstawowe: 0, Średnie: 0
fav_animals - Psy: 18, Inne: 6, Koty: 6, Brak ulubionych: 1, Koty i Psy: 0
fav_place - W lesie: 18, W górach: 11, Inne: 2, Nad wodą: 0
gender - Mężczyzna: 27, K

In [9]:
response = openai_client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
)

In [10]:
# zamiana odpowiedzi llm na taką, która jest akceptowalna dla json loads
result = response.choices[0].message.content.replace("```json", "").replace("```", "").strip()
cluster_names_and_descriptions = json.loads(result)

In [11]:
# zapisywanie do pliku
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "w") as f:
    f.write(json.dumps(cluster_names_and_descriptions))

In [12]:
# odczytywanie z pliku
with open("welcome_survey_cluster_names_and_descriptions_v1.json", "r") as f:
    print(json.loads(f.read()))

{'Cluster 0': {'name': 'Miłośnicy Lasów i Psów', 'description': 'W tym klastrze znajdują się osoby, które preferują spędzanie czasu w lesie i mają wyższe wykształcenie. Większość z nich to mężczyźni w wieku 45-54 lat, a ich ulubionymi zwierzętami są psy.'}, 'Cluster 1': {'name': 'Profesjonaliści Nad Wodą', 'description': 'W tym klastrze znajdują się osoby z wyższym wykształceniem, które preferują spędzanie czasu nad wodą. Większość z nich to mężczyźni w wieku 35-44 lat, a ich ulubionymi zwierzętami są psy.'}, 'Cluster 2': {'name': 'Młodzi Miłośnicy Gór', 'description': 'W tym klastrze znajdują się młodsze osoby z wykształceniem średnim, które preferują spędzanie czasu w górach. Większość z nich to mężczyźni w wieku 25-34 lat, a ich ulubionymi zwierzętami są psy.'}, 'Cluster 3': {'name': 'Wykształceni Miłośnicy Gór', 'description': 'W tym klastrze znajdują się osoby z wyższym wykształceniem, które preferują spędzanie czasu w górach. Większość z nich to mężczyźni w wieku 35-44 lat, a ich